In [1]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime

from scipy.stats import linregress


In [2]:
# import data and put in dataframe
CapitalBike202104= "data/202104-citibike-tripdata.csv"
CapitalBike202105= "data/202105-citibike-tripdata.csv"
CapitalBike202106= "data/202106-citibike-tripdata.csv"

CapitalBike202104_df = pd.read_csv(CapitalBike202104)
CapitalBike202105_df = pd.read_csv(CapitalBike202105)
CapitalBike202106_df = pd.read_csv(CapitalBike202106)


C:\Users\rod\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\rod\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# verify read_csv
CapitalBike202104_df.tail()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
2067664,DE4F31703A6FBD72,docked_bike,2021-04-18 15:48:21,2021-04-18 16:03:42,Hudson St & W 13 St,6115.06,Mercer St & Spring St,5532.01,40.740057,-74.005274,40.723627,-73.999496,casual
2067665,391040FC8CAF2601,docked_bike,2021-04-19 19:39:00,2021-04-19 19:55:18,W 87 St & Amsterdam Ave,7458.03,3 Ave & E 62 St,6762.04,40.788390,-73.974700,40.763126,-73.965269,member
2067666,1DB5175D0F873C53,docked_bike,2021-04-21 11:57:03,2021-04-21 12:15:49,E 27 St & 1 Ave,6004.06,3 Ave & E 62 St,6762.04,40.739445,-73.976806,40.763126,-73.965269,member
2067667,3004043A978641B3,docked_bike,2021-04-04 14:41:23,2021-04-04 14:56:46,Myrtle Ave & St Edwards St,4659.02,Brooklyn Bridge Park - Pier 2,4756.04,40.693270,-73.977038,40.698458,-73.997178,member
2067668,E74E90176B0527D8,docked_bike,2021-04-23 18:40:57,2021-04-23 19:22:18,Hudson St & W 13 St,6115.06,W 18 St & 9 Ave,6190.03,40.740057,-74.005274,40.743534,-74.003676,casual


In [4]:
# join all df together
combined_df = pd.concat([CapitalBike202104_df, CapitalBike202105_df, CapitalBike202106_df], ignore_index=True)

In [5]:
combined_df.count()

ride_id               7969351
rideable_type         7969351
started_at            7969351
ended_at              7969351
start_station_name    7968983
start_station_id      7968983
end_station_name      7950073
end_station_id        7950073
start_lat             7969351
start_lng             7969351
end_lat               7952330
end_lng               7952330
member_casual         7969351
dtype: int64

In [6]:
combined_df.shape


(7969351, 13)

In [7]:
# clean data
combined_df.dropna(inplace=True)
combined_df.drop(['rideable_type'],axis=1,inplace=True)


In [8]:
combined_df.count()

ride_id               7950063
started_at            7950063
ended_at              7950063
start_station_name    7950063
start_station_id      7950063
end_station_name      7950063
end_station_id        7950063
start_lat             7950063
start_lng             7950063
end_lat               7950063
end_lng               7950063
member_casual         7950063
dtype: int64

In [9]:
combined_df.shape

(7950063, 12)

In [10]:
# create a duration column in seconds from starting and end times 

combined_df['Duration'] = pd.to_timedelta(pd.to_datetime(combined_df['ended_at'], format='%Y-%m-%d %H:%M:%S')-pd.to_datetime(combined_df['started_at'], format='%Y-%m-%d %H:%M:%S')).dt.total_seconds()
#combined_2020_2ndcol_df = combined_2020_2nd_df[['Duration','started_at','ended_at','start_station_id','start_station_name','end_station_id','end_station_name','ride_id','member_casual']]


In [11]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7950063 entries, 0 to 7969350
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   started_at          object 
 2   ended_at            object 
 3   start_station_name  object 
 4   start_station_id    object 
 5   end_station_name    object 
 6   end_station_id      object 
 7   start_lat           float64
 8   start_lng           float64
 9   end_lat             float64
 10  end_lng             float64
 11  member_casual       object 
 12  Duration            float64
dtypes: float64(5), object(8)
memory usage: 849.2+ MB


In [12]:
# separate out start datetime format
combined_df['DateTime'] =  pd.to_datetime(combined_df['started_at'], format='%Y-%m-%d %H:%M:%S')
# extracting time from timestamp
combined_df['Time'] = combined_df['DateTime'].dt.strftime('%H:%M')
combined_df['Month'] = combined_df['DateTime'].dt.strftime('%b')
combined_df['Date'] = combined_df['DateTime'].dt.strftime('%x')


In [13]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7950063 entries, 0 to 7969350
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   started_at          object        
 2   ended_at            object        
 3   start_station_name  object        
 4   start_station_id    object        
 5   end_station_name    object        
 6   end_station_id      object        
 7   start_lat           float64       
 8   start_lng           float64       
 9   end_lat             float64       
 10  end_lng             float64       
 11  member_casual       object        
 12  Duration            float64       
 13  DateTime            datetime64[ns]
 14  Time                object        
 15  Month               object        
 16  Date                object        
dtypes: datetime64[ns](1), float64(5), object(11)
memory usage: 1.1+ GB


In [14]:
# separate out end datetime format
##combined_df['e_DateTime'] =  pd.to_datetime(combined_df['ended_at'], format='%Y-%m-%d %H:%M:%S')
# extracting time from timestamp
##combined_df['e_Time'] = combined_df['e_DateTime'].dt.strftime('%H:%M')

In [15]:
# change column header names to more readable format
combined_df = combined_df.rename(columns={'started_at':'start date','ended_at':'end date','start_station_id':'start station number','start_station_name':'start station','end_station_id':'end station number','end_station_name':'end station','start_lat':'start latitude','start_lng':'start longitude','end_lat':'end latitude','end_lng':'end longitude','Time':'time','Month':'month','Date':'date','member_casual':'member type','Duration':'duration seconds'})
combined_df.tail()

,ride_id,start date,end date,start station,start station number,end station,end station number,start latitude,start longitude,end latitude,end longitude,member type,duration seconds,DateTime,time,month,date
7969346,5CCF02508D6C4209,2021-06-30 13:13:05,2021-06-30 13:25:54,Alexander Ave & E 134 St,7712.04,St. Nicholas Ave & W 126 St,7756.10,40.807466,-73.927107,40.811432,-73.951878,casual,769.0,2021-06-30 13:13:05,13:13,Jun,06/30/21
7969347,C4D61E2B06185BC1,2021-06-23 22:51:46,2021-06-23 22:56:56,8 Ave & W 31 St,6450.05,W 20 St & 8 Ave,6224.05,40.750585,-73.994685,40.743453,-74.000040,member,310.0,2021-06-23 22:51:46,22:51,Jun,06/23/21
7969348,B58C37145FC92C4B,2021-06-19 16:18:58,2021-06-19 16:33:32,S Portland Ave & Hanson Pl,4354.05,Eastern Pkwy & Washington Ave,3928.08,40.685396,-73.974315,40.671649,-73.963115,casual,874.0,2021-06-19 16:18:58,16:18,Jun,06/19/21
7969349,C166DBE51A54FDAE,2021-06-23 11:29:37,2021-06-23 11:30:59,8 Ave & W 31 St,6450.05,8 Ave & W 33 St,6450.12,40.750585,-73.994685,40.751551,-73.993934,member,82.0,2021-06-23 11:29:37,11:29,Jun,06/23/21
7969350,DB56BB2AE0570263,2021-06-19 00:54:21,2021-06-19 01:02:39,8 Ave & W 31 St,6450.05,W 20 St & 8 Ave,6224.05,40.750585,-73.994684,40.743453,-74.000040,member,498.0,2021-06-19 00:54:21,00:54,Jun,06/19/21


In [17]:
# convert seconds to minutes
combined_df['duration minutes'] = combined_df['duration seconds']/60
combined_df.head()

,ride_id,start date,end date,start station,start station number,end station,end station number,start latitude,start longitude,end latitude,end longitude,member type,duration seconds,DateTime,time,month,date,duration minutes
0,7099F9531C9DA16D,2021-04-28 17:27:24,2021-04-28 17:34:31,DeKalb Ave & Hudson Ave,4513.06,Douglass St & 4 Ave,4175.14,40.689888,-73.981013,40.679279,-73.981540,member,427.0,2021-04-28 17:27:24,17:27,Apr,04/28/21,7.116667
1,EDC6F1B1A69A7863,2021-04-27 17:28:02,2021-04-27 17:33:07,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member,305.0,2021-04-27 17:28:02,17:28,Apr,04/27/21,5.083333
2,431A76830E10408A,2021-04-28 18:05:19,2021-04-28 18:10:48,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member,329.0,2021-04-28 18:05:19,18:05,Apr,04/28/21,5.483333
3,C00248A0ADC009FA,2021-04-20 21:35:47,2021-04-20 21:56:31,Irving Ave & DeKalb Ave,4898.02,Division Ave & Hooper St,5045.05,40.702700,-73.920950,40.706842,-73.954435,casual,1244.0,2021-04-20 21:35:47,21:35,Apr,04/20/21,20.733333
4,BE78810D746EE516,2021-04-30 00:34:54,2021-04-30 01:03:29,3 Ave & E 112 St,7543.15,E 115 St & Lexington Ave,7599.09,40.795508,-73.941606,40.797911,-73.942300,casual,1715.0,2021-04-30 00:34:54,00:34,Apr,04/30/21,28.583333


In [18]:
# write dataframe to csv
combined_df.to_csv('combined_data.csv', index=False)